In [ ]:
import os
args = os.getenv("ARGS", "release real").split(" ")
release = args[0]
arch = args[1]

In [ ]:
try:
    import BACKEND_LIBRARY_IN  # noqa: F401
except ImportError:
    import firedrake
else:
    try:
        import firedrake
    except ImportError:
        if release != "development":
            if arch != "complex":
                !wget "https://PROJECT_NAME_IN.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
            else:
                !wget "https://PROJECT_NAME_IN.github.io/releases/firedrake-install-release-complex.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        else:
            if arch != "complex":
                !wget "https://PROJECT_NAME_IN.github.io/releases/firedrake-install-development-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
            else:
                !wget "https://PROJECT_NAME_IN.github.io/releases/firedrake-install-development-complex.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        import firedrake  # noqa: F401

In [ ]:
try:
    import irksome
except ImportError:
    !python3 -m pip install --no-dependencies git+https://github.com/firedrakeproject/Irksome.git
    import irksome  # noqa: F401

In [ ]:
from ufl.algorithms.ad import expand_derivatives
from firedrake import *
from irksome import *

In [ ]:
N = 20
x0 = 0.0
x1 = 10.0
y0 = 0.0
y1 = 10.0

In [ ]:
msh = RectangleMesh(N, N, x1, y1)
V = FunctionSpace(msh, "CG", 1)

In [ ]:
MC = MeshConstant(msh)
dt = MC.Constant(1.0 / N)
t = MC.Constant(0.0)

In [ ]:
x, y = SpatialCoordinate(msh)
S = Constant(2.0)
C = Constant(1000.0)
B = (x - Constant(x0)) * (x - Constant(x1)) * (y - Constant(y0)) * (y - Constant(y1)) / C
R = (x * x + y * y) ** 0.5
uexact = B * atan(t) * (pi / 2.0 - atan(S * (R - t)))
rhs = expand_derivatives(diff(uexact, t)) - div(grad(uexact))

In [ ]:
u = interpolate(uexact, V)
v = TestFunction(V)
F = inner(Dt(u), v) * dx + inner(grad(u), grad(v)) * dx - inner(rhs, v) * dx
bc = DirichletBC(V, 0, "on_boundary")

In [ ]:
butcher_tableau = GaussLegendre(1)
luparams = {"mat_type": "aij", "ksp_type": "preonly", "pc_type": "lu"}
stepper = TimeStepper(F, butcher_tableau, t, dt, u, bcs=bc, solver_parameters=luparams)

In [ ]:
while float(t) < 1.0:
    if float(t) + float(dt) > 1.0:
        dt.assign(1.0 - float(t))
    print("solving for time", float(t) + float(dt))
    stepper.advance()
    t.assign(float(t) + float(dt))
print(norm(u - uexact) / norm(uexact))